Importación de librerías a utilizar

In [1]:
import pandas as pd
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

Creación del DataFrame

In [2]:
df_g=pd.read_csv(r'C:\Users\Nahir\Desktop\CSV\output_steam_games.csv', low_memory=False)
df_g.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Exploración del DataFarme

In [3]:
df_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  object 
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(1), object(12)
memory usage: 11.9+ MB


In [4]:
#Devuelve cuantos regsitros duplicados hay
df_g.duplicated().sum()

88309

In [3]:
#Se borran los duplicados
df_g = df_g.drop_duplicates()
#Se viasualiza como quedó el DataFrame
df_g.head() 


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域


In [4]:
#Se eliminan los nulos que quedaron
df_g = df_g.dropna()
#Se eliminan las columnas innecesarias
df_g=df_g.drop(['publisher','title','url','release_date','reviews_url','price','early_access','developer'], axis=1) 
#Se visualiza como quedó el DataFrame
df_g.head() 

,genres,app_name,tags,specs,id
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],761140.0
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",643980.0
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...","['Single-player', 'Multi-player', 'Online Mult...",670290.0
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,"['Action', 'Adventure', 'Casual']",['Single-player'],767400.0
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,"['Action', 'Adventure', 'Simulation', 'FPS', '...","['Single-player', 'Steam Achievements']",772540.0


In [5]:
#Se reinicia el índice del DataFrame
df_g.reset_index(drop=True, inplace=True)
#Se observa como quedó el DatFrame
df_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22511 entries, 0 to 22510
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   genres    22511 non-null  object 
 1   app_name  22511 non-null  object 
 2   tags      22511 non-null  object 
 3   specs     22511 non-null  object 
 4   id        22511 non-null  float64
dtypes: float64(1), object(4)
memory usage: 879.5+ KB


Machine Learning

In [6]:
#Se crea una lista de columnas relevantes
columnas_similares = ['genres', 'tags', 'specs']
#Se crea una columna "variables_juntas" con las listas combinadas en cadenas
df_g['variables_juntas'] = df_g[columnas_similares].apply(lambda row: ' '.join(row), axis=1)

In [7]:
#Se crea la matriz TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_g['variables_juntas'])

In [13]:
#Se reduce la dimensionalidad con Truncated SVD
num_components = 100  # Número de componentes a mantener
svd = TruncatedSVD(n_components=num_components)
svd_matrix = svd.fit_transform(tfidf_matrix)

In [9]:
#Se calcula la similitud del coseno en la matriz reducida
#cosine_sim_svd = cosine_similarity(svd_matrix, svd_matrix)

PKL

In [9]:
pkl_file = open('svd_matrix.pkl', 'ab')
pickle.dump(svd_matrix, pkl_file)
pkl_file.close()

In [15]:
pkl_file = open('games_ml.pkl', 'ab')
pickle.dump(df_g, pkl_file)
pkl_file.close()

In [14]:
# Función para obtener recomendaciones
def get_recommendations(id, svd_matrix, df, top_n=5):
    game_index = df[df['id'] == id].index[0]
    #Se calcula la similitud del coseno en la matriz reducida
    cosine_sim_svd = cosine_similarity(svd_matrix, svd_matrix)
    similar_games = list(enumerate(cosine_sim_svd[game_index]))
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)
    similar_games = similar_games[1:top_n+1]
    similar_game_indices = [i[0] for i in similar_games]
    recommended_games = df['app_name'].iloc[similar_game_indices]
    return recommended_games

In [15]:
# Obtener recomendaciones para un juego específico
input_game = 761140.0
recommendations = get_recommendations(input_game, svd_matrix, df_g)

print(f"Recomendaciones para '{input_game}':")
print(recommendations)

Recomendaciones para '761140.0':
7763               Trivia Vault: 1980's Trivia 2
7891    Trivia Vault: Science & History Trivia 2
7982        Crankies Workshop: Grizzbot Assembly
7992         Trivia Vault: Super Heroes Trivia 2
8004         Crankies Workshop: Bozzbot Assembly
Name: app_name, dtype: object
